<a href="https://colab.research.google.com/github/NckolasGomes6696/My-stuff/blob/main/126696_cart.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import gym
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers
import matplotlib.pyplot as plt
import moviepy.editor as mpy

# Definição do ambiente (CartPole)
env = gym.make('CartPole-v1')
num_actions = env.action_space.n  # Ações possíveis (esquerda/direita)

# Modelo de Política (Actor)
def create_actor_model():
    inputs = layers.Input(shape=(env.observation_space.shape[0],))
    hidden_layer = layers.Dense(128, activation='relu')(inputs)
    hidden_layer = layers.Dense(128, activation='relu')(hidden_layer)
    output = layers.Dense(num_actions, activation='softmax')(hidden_layer)
    model = tf.keras.Model(inputs=inputs, outputs=output)
    return model

# Modelo Crítico (Critic)
def create_critic_model():
    inputs = layers.Input(shape=(env.observation_space.shape[0],))
    hidden_layer = layers.Dense(128, activation='relu')(inputs)
    hidden_layer = layers.Dense(128, activation='relu')(hidden_layer)
    output = layers.Dense(1)(hidden_layer)
    model = tf.keras.Model(inputs=inputs, outputs=output)
    return model

# Inicializando os modelos
actor_model = create_actor_model()
critic_model = create_critic_model()

# Definir otimizadores
optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)
huber_loss = tf.keras.losses.Huber()

# Função para calcular o reforço (reward) futuro esperado e atualizar os modelos
def update_actor_critic(states, actions, advantages, returns):
    with tf.GradientTape() as tape:
        action_probs = actor_model(states, training=True)
        critic_value = critic_model(states, training=True)

        # Cálculo da perda de política (actor)
        action_one_hot = tf.one_hot(actions, num_actions)
        selected_action_probs = tf.reduce_sum(action_probs * action_one_hot, axis=1)
        log_action_probs = tf.math.log(selected_action_probs + 1e-10)
        actor_loss = -tf.reduce_sum(log_action_probs * advantages)

        # Cálculo da perda do crítico
        #critic_value = tf.squeeze(critic_value)  # Remover dimensões desnecessárias - This line is causing the error, since it converts critic_value to a scalar
        #returns = tf.squeeze(returns)  # Remover dimensões desnecessárias - This line is causing the error, since it converts returns to a scalar
        critic_loss = huber_loss(returns, critic_value)

        total_loss = actor_loss + critic_loss

    # Computar e aplicar gradientes
    grads = tape.gradient(total_loss, actor_model.trainable_variables + critic_model.trainable_variables)
    optimizer.apply_gradients(zip(grads, actor_model.trainable_variables + critic_model.trainable_variables))

# Função de treino
def train(env, max_episodes=1000, gamma=0.99):
    reward_history = []
    for episode in range(max_episodes):
        state = env.reset()
        state = np.reshape(state, [1, env.observation_space.shape[0]])

        episode_reward = 0
        done = False

        while not done:
            # Prever ação do ator
            action_probs = actor_model(state, training=False)
            action_probs = np.squeeze(action_probs)  # Garantir a forma correta
            action = np.random.choice(num_actions, p=action_probs)

            # Realizar a ação
            next_state, reward, done, _ = env.step(action)
            next_state = np.reshape(next_state, [1, env.observation_space.shape[0]])

            # Calcular valor do crítico
            critic_value = critic_model(state)
            next_critic_value = critic_model(next_state)

            # Atualizar recompensa acumulada
            episode_reward += reward

            # Calcular o valor-alvo e vantagem
            if done:
                target = reward
            else:
                target = reward + gamma * np.squeeze(next_critic_value)

            advantage = target - np.squeeze(critic_value)

            # Atualizar os modelos
            update_actor_critic(
                tf.convert_to_tensor(state, dtype=tf.float32),
                np.array([action]),
                np.array([advantage]),
                np.array([target])
            )

            state = next_state

        # Salvar a recompensa do episódio
        reward_history.append(episode_reward)
        print(f'Episódio {episode + 1}/{max_episodes} - Recompensa: {episode_reward}')

    return reward_history

# Treinamento do A2C
reward_history = train(env)

# Plotando a curva de aprendizagem
plt.plot(reward_history)
plt.xlabel('Episódio')
plt.ylabel('Recompensa')
plt.title('Curva de Aprendizado')
plt.show()

# Função para gravar o vídeo de um episódio
def record_video(env, model, video_path="cartpole_a2c.mp4"):
    images = []
    state = env.reset()
    done = False

    while not done:
        img = env.render(mode='rgb_array')
        images.append(img)
        action_probs = model(np.reshape(state, [1, env.observation_space.shape[0]]))
        action_probs = np.squeeze(action_probs)  # Garantir a forma correta
        action = np.argmax(action_probs)
        state, _, done, _ = env.step(action)

    env.close()

    # Criar e salvar o vídeo
    clip = mpy.ImageSequenceClip(images, fps=30)
    clip.write_videofile(video_path)

# Gravar um episódio
record_video(env, actor_model)

  from scipy.ndimage.filters import sobel

  deprecation(

  deprecation(

  if not isinstance(terminated, (bool, np.bool8)):



Episódio 1/1000 - Recompensa: 13.0
Episódio 2/1000 - Recompensa: 21.0
Episódio 3/1000 - Recompensa: 11.0
Episódio 4/1000 - Recompensa: 10.0
Episódio 5/1000 - Recompensa: 34.0
Episódio 6/1000 - Recompensa: 11.0
Episódio 7/1000 - Recompensa: 23.0
Episódio 8/1000 - Recompensa: 8.0
Episódio 9/1000 - Recompensa: 10.0
Episódio 10/1000 - Recompensa: 8.0
Episódio 11/1000 - Recompensa: 9.0
Episódio 12/1000 - Recompensa: 10.0
Episódio 13/1000 - Recompensa: 9.0
Episódio 14/1000 - Recompensa: 10.0
Episódio 15/1000 - Recompensa: 9.0
Episódio 16/1000 - Recompensa: 10.0
Episódio 17/1000 - Recompensa: 10.0
Episódio 18/1000 - Recompensa: 8.0
Episódio 19/1000 - Recompensa: 9.0
Episódio 20/1000 - Recompensa: 9.0
Episódio 21/1000 - Recompensa: 11.0
Episódio 22/1000 - Recompensa: 9.0
Episódio 23/1000 - Recompensa: 9.0
Episódio 24/1000 - Recompensa: 8.0
Episódio 25/1000 - Recompensa: 10.0
Episódio 26/1000 - Recompensa: 10.0
Episódio 27/1000 - Recompensa: 9.0
Episódio 28/1000 - Recompensa: 8.0
Episódio 29/10